In [1]:
# Loading required libraries
import csv
import sys
import pandas as pd
import os
import glob
import itertools
import numpy as np
from collections import Counter
import copy
import math
import random
import time


# Loading trace : Needs to expanded into 4K chunks
path = r'/Users/chandranilchakraborttii/Documents/GC_pred/data'
all_files = glob.glob(os.path.join(path, "synthetic_dataprep_B_with_deathtime_new_1.0GB_SSD.csv"))

f = all_files[0]  # Change the file name as required
print("File " + str(f))
df = pd.read_csv(f,engine='python',skiprows =1,header=None,na_values=['-1'], index_col=False)
cols = ['LBA','deathtime']
df.columns = cols
print("Before",len(df))
df['deathtime'] = df['deathtime'].replace(np.NaN, -1)
df = df.loc[df['deathtime'] != -1]
df = df.loc[df['deathtime'] != 1]
print("After",len(df))
lba_list = df['LBA'].tolist()
deathtime_list = df['deathtime'].tolist()
print("Min LBA in the dataset :", min(lba_list))
print("Max LBA in the dataset :", max(lba_list))
print("Number of unique LBAs in the data :",len(Counter(df['LBA'])))
print("Number of IO Accesses :",len(df))

File /Users/chandranilchakraborttii/Documents/GC_pred/data/synthetic_dataprep_B_with_deathtime_new_1.0GB_SSD.csv
Before 1048580
After 786430
Min LBA in the dataset : 0
Max LBA in the dataset : 262144
Number of unique LBAs in the data : 262145
Number of IO Accesses : 786430


In [2]:
print (len(df))
df = df.dropna()
df = df.reset_index(drop=True)
print (len(df))
df = df[df.deathtime != -1]
print (len(df))
df = df[df.deathtime != 0]
print (len(df))
df = df[df.deathtime != 1]
print (len(df))

786430
786430
786430
786430
786430


In [3]:
# import random
# new_lba_list = []
# new_dt_list = []

# extent = 5
# counter = 0
# new_dt = random.randint(0,19999)
# while counter < len(lba_list):
#     new_lba = random.randint(1,5) * lba_list[counter]
#     new_lba_list.append(new_lba)
#     extent = extent - 1
#     if(extent < 1):
#         if(counter % 2 == 0 or counter%3 == 0 or counter % 5 == 0 or counter % 7 == 0 ):
#             new_dt = random.randint(0,500)
#         else:
#             new_dt = random.randint(0,19999)
#     extent = random.randint(5,15)
#     new_dt_list.append(new_dt)
#     counter = counter + 1

# print(len(new_lba_list))
# print(len(new_dt_list))

# lba_list = new_lba_list
# deathtime_list = new_dt_list
# df['lba'] = lba_list
# df['death_time']  = deathtime_list

In [177]:
#SSD specifications
num_page_addresses = len(Counter(lba_list))
page_size = 4096
page_per_block = 64    
GB = 1024*1024*1024
SSD_size = num_page_addresses*page_size
# SSD_size_GB_normal = round_decimals_up(SSD_size/GB,1)
SSD_size_GB_normal = SSD_size/GB
over_provisioning_ratio = 0.2
LOG_PAGE_PER_BLOCK = int(math.log(page_per_block,2))
# SSD_size_full = round_decimals_up((1 + over_provisioning_ratio)*SSD_size_GB_normal,1)
SSD_size_full = (1 + over_provisioning_ratio)*SSD_size_GB_normal
print("SSD Capacity (Available in GB) :",SSD_size_GB_normal)
print("SSD Capacity (Total in GB)     :",SSD_size_full)

SSD Capacity (Available in GB) : 1.0000038146972656
SSD Capacity (Total in GB)     : 1.2000045776367188


In [178]:
import copy

GB = 1024*1024*1024
ssd_capacity = SSD_size_full *GB

# Make the block,page and physical addresses for normal and Overprovisioned capacity
page_addresses = []
block_addresses = []
block_placement = 0
start_counter = -1
block_addresses.append(0)

while(start_counter < (ssd_capacity/page_size) - page_size):
    start_counter = start_counter + 1
    page_addresses.append(int(start_counter))
    if(block_placement >= page_per_block):
        block_addresses.append(int(start_counter))
        block_placement = 0

    block_placement = block_placement + 1

free_list_block = copy.deepcopy(block_addresses)
free_list_page = copy.deepcopy(page_addresses)

block_struct = {}
for x in free_list_block:
    start_lba = x
    write_ptr=0
    invalid_pages=0
    death_time = 0
    valid_bitmap = []
    priority = False
    death_time_original = 0
    for x in range(page_per_block):
        valid_bitmap.append(False)

    segment = [start_lba,invalid_pages,valid_bitmap,write_ptr,death_time,priority,death_time_original]
    block_struct[start_lba]=segment
    

print("Total number of Blocks created: ", len(block_addresses))
print("Total number of Pages created:  ", len(page_addresses))
print(str(len(block_struct)) + " 4K blocks Initialized" )

Total number of Blocks created:  4852
Total number of Pages created:   310479
4852 4K blocks Initialized


In [179]:
def invalidate_lba(lba):
    prev = L2P[lba]
    prev_block = (prev >> LOG_PAGE_PER_BLOCK)*page_per_block
    prev_page = prev % page_per_block
    block_details = block_struct[prev_block]                                 # Getting block details
    block_struct[prev_block][2][prev_page] = False                          # Setting bitmap to False
    block_struct[prev_block][1] = block_struct[prev_block][2].count(True)     # Setting invalid pages
    L2P.pop(lba)


    
#map LBA to phys
def map_lba(lba,deathtime, block_IO_burst,in_gc):
    # Finding which block to add the LBA
    for x in block_IO_burst:
        if(block_struct[x][3] == page_per_block):
            block_IO_burst = check_GC(block_IO_burst,in_gc)
            
    block_map={}
    found = False
    for x in block_IO_burst:
        # If the death Time has passed, make it priority 
        if(block_struct[x][5] == True):
            block_select = x
            found = True 
        else:
            block_map[x]= block_struct[x][4]
            
    # Finding the block with closest death time 
    if(found !=True):
        delta = max(deathtime_range_list)*100
        keys = list(block_map.keys())
        block_select = -1
        for x in keys:
            tmp = abs(block_map[x] - deathtime)
            if(tmp < delta):
                delta = tmp
                block_select = x

    if(block_struct[block_select][5] == True):
        priority_writes[0] = priority_writes[0] + 1

    # print(block_select)
    # Block Found, now updating block          
    phys_addr = block_struct[block_select][0] + (block_struct[block_select][3])
    L2P[lba] = phys_addr
    P2L[phys_addr] = lba   
    block_struct[block_select][2][block_struct[block_select][3]] = True             # Setting Bitmap
    block_struct[block_select][1] = block_struct[block_select][2].count(True)            # Setting invalid pages
    block_struct[block_select][3] = block_struct[block_select][3] + 1               # Increasing Write pointer
    total_user_writes[0] = total_user_writes[0] + 1
    if(in_gc == True):
        total_gc_writes[0] = total_gc_writes[0] + 1

    

#check if we need to close/open block. Do not perform GC if we are already
def check_GC (block_IO_burst, in_gc):
    for x in block_IO_burst:
        # If block is full, close block and reset death time
        if(block_struct[x][3] == page_per_block):
            death_time = block_struct[x][6]               # Copying original death time to be set in the new block
            closed_blocks.append(x)                       # Adding to closed list
            block_IO_burst.remove(x)                    
            new_block = free_list_block.pop(0)            #  Requesting a new block
            block_IO_burst.append(new_block)              # Adding to open blocks
            write_ptr=0
            invalid_pages=0
            valid_bitmap = []
            priority = False
            # print(block_IO_burst)
            for x in range(page_per_block):
                valid_bitmap.append(False)

            block_struct[new_block][1] = invalid_pages
            block_struct[new_block][2] = valid_bitmap
            block_struct[new_block][3] = write_ptr
            block_struct[new_block][4] = death_time       # Setting death time counter of the new block
            block_struct[new_block][6] = death_time       # Setting death time of the new block
    
    if(len(free_list_block) == 0):
        print("FAIL WHILE DOING GC, RAN OUT OF BLOCKS") 
   # Checking if GC is needed
    elif (len(free_list_block) <= GC_THRESHOLD):
        # Checking if GC is already going on
        if(in_gc != True):
            in_gc = do_greedy_gc(block_IO_burst,in_gc) 
    return block_IO_burst



def do_greedy_gc(block_IO_burst,in_gc):
    # print("Here")
    in_gc = True
    min_val = float('inf')
    for x in closed_blocks:              
        if (block_struct[x][1] < min_val):
            min_val = block_struct[x][1]
            gc_blk = x
        # For each closed block, check phys_addr: If valid bitmap is True (data is valid), copy to OP capacity
    closed_blocks.remove(gc_blk)
    for pg in range(page_per_block):
        #figure out the logical addresses for all phys pages in the gc block
        phys_addr = block_struct[gc_blk][0] + pg
        # Updating P2L
        if (phys_addr in P2L):
            gc_lba = P2L[phys_addr]
            P2L.pop(phys_addr)            
        # Updating L2P
        # Checking for valid bitmap
        prev_block = (phys_addr >> LOG_PAGE_PER_BLOCK)*page_per_block
        prev_page = phys_addr % page_per_block
        bitmap = block_struct[prev_block][2][prev_page]
        death_time = block_struct[gc_blk][6]
        # If valid bitmap is True (data is valid), copy to OP capacity, increase GC writes
        if (bitmap == True):
            invalidate_lba(gc_lba)
            #check if we need to get a new block
            block_IO_burst = check_GC(block_IO_burst,in_gc)
            #move the gc'ed block t-o a new location
            map_lba(gc_lba,death_time,block_IO_burst,in_gc)
            block_IO_burst = decrease_death_time(block_IO_burst)  
    

    death_time = block_struct[prev_block][6]                      # Getting current death_time for the block
    free_list_block.append(gc_blk) 
    if(gc_writes > 64):
        print("GC writes not as expected", gc_writes)
        
    in_gc = False
    return in_gc

def decrease_death_time(block_IO_burst):
    # Decreasing Death Time for each block which has atleast one write
    for x in block_IO_burst:
        if (block_struct[x][4] <= 0 and block_struct[x][5] != True):
            candidate = x
            dt_original = block_struct[candidate][6]    
            write_ptr = block_struct[candidate][3]
            dt_to_increase = ((page_per_block - write_ptr)/100)+2
            block_struct[candidate][4] = int(dt_to_increase)
            block_struct[candidate][5] = True
        if (block_struct[x][3] != 0):
            block_struct[x][4] = block_struct[x][4] - 1
    return block_IO_burst



In [180]:
# Setting global parameters
# Initalizing Starting Free Blocks..
global gc_writes
global in_gc 

num_cur_blocks_open = 500       # Hyperparameter

L2P = {}
P2L = {}
closed_blocks = []
cur_blocks_open = []
lba_burst = []
deathtime_range_list = []
interval = float(100/num_cur_blocks_open)
gc_writes = 0
in_gc = False
death_time_passed = []

for x in range(num_cur_blocks_open):
    deathtime_range_list.append(int(np.percentile(deathtime_list, (x+1)*interval)))


block_IO_burst = []
death_time_ranges = []
print("Initalizing Starting Free Blocks...")
for x in range(num_cur_blocks_open):
    block_num = free_list_block.pop(0)                                            # Getting a free block
    block_IO_burst.append(block_num) 
    block_struct[block_num][4] = deathtime_range_list[x]                          # Setting death time
    block_struct[block_num][6] = deathtime_range_list[x]  

Initalizing Starting Free Blocks...


In [181]:
import time
total_gc_writes = []
total_gc_writes.append(0)
priority_writes = []
priority_writes.append(0)
total_user_writes = []
total_user_writes.append(0)
total_user_writes.append(0)
blocks_closed_early = []
counter = 0
GC_THRESHOLD = 20
in_gc = False

print("Starting Trace..!")
start_time = time.time()
for x in range(1):
  counter = 0
  while(counter < len(lba_list)):
      if(counter >100000 and counter%100000==0):
          print("Percentage completed in (%)  :", (counter/len(lba_list))*100)
      lba = int(lba_list[counter]) + 1
      death_time = int(deathtime_list[counter])
      if lba in L2P:
          invalidate_lba(lba)
      block_IO_burst = check_GC(block_IO_burst,in_gc)
      map_lba(lba,death_time,block_IO_burst,in_gc)
      block_IO_burst = decrease_death_time(block_IO_burst)
      counter = counter + 1
    

end_time = time.time()
run_time = end_time - start_time
print("Trace Replay Ended!")

Starting Trace..!
Percentage completed in (%)  : 25.43137977950994
Percentage completed in (%)  : 38.1470696692649
Percentage completed in (%)  : 50.86275955901988
Percentage completed in (%)  : 63.57844944877484
Percentage completed in (%)  : 76.2941393385298
Percentage completed in (%)  : 89.00982922828477


IndexError: list assignment index out of range

In [ ]:


# total_gc_writes = []
# total_gc_writes.append(0)
# priority_writes = []
# priority_writes.append(0)
# total_user_writes = []
# total_user_writes.append(0)

# GC_THRESHOLD = 100
# over_provisioning_ratio = 0.2
# num_cur_blocks_open = 100


# total_gc_writes[0] = 338490
# total_user_writes[0] = 786430 + total_gc_writes[0]
# priority_writes[0] = 9119
# run_time = 34.3

In [182]:
print("GC Threshold: ",GC_THRESHOLD)
print("OverProvisioning Ratio: ",over_provisioning_ratio)
print("Number of Open Blocks: ",num_cur_blocks_open)
print("Total Number of writes: ",total_user_writes[0])
print("Total Number of GC writes: ",total_gc_writes[0])
print("Total Number of Priority writes: ",priority_writes[0])
print("Execution Time for the FTL :",run_time)
user_writes = (total_user_writes[0] - total_gc_writes[0])
print("Write Amplification:  ",(total_user_writes[0]/user_writes))
print("Total Number of user writes:",(total_user_writes[0] - total_gc_writes[0]))
print("Done")

GC Threshold:  20
OverProvisioning Ratio:  0.2
Number of Open Blocks:  500
Total Number of writes:  1455198
Total Number of GC writes:  751043
Total Number of Priority writes:  1167640
Execution Time for the FTL : 150.45615005493164
Write Amplification:   2.066587612102449
Total Number of user writes: 704155
Done
